In [4]:
import numpy as np
from numba import jit


@jit(nopython=True)
def makeRandomCouplingMatrix(nspins,ncolors,sparsity):
    """
    Creates random Potts coupling matrix with specified sparsity 
    """
    
    J = np.zeros((nspins,nspins,ncolors,ncolors))
    numEntries = int(sparsity*J.size)
    numFilled = 0
    while numFilled < numEntries:
            m,n = np.random.choice(nspins,2)
            a,b = np.random.choice(ncolors,2)

            if J[m,n,a,b]==0 and J[n,m,b,a]==0 and m!=n:
                    J[m,n,a,b] = 2*np.random.rand()-1
                    J[n,m,b,a] = J[m,n,a,b]
                    numFilled += 2
                    
            if m==n and a==b:
                J[m,n,a,b] = 2*np.random.rand()-1
                numFilled += 1

    return J


@jit(nopython=True)
def conditionalPottsProb(i,x,J,nspins,ncolors,beta=1,xfmt=0):
    """
    Returns the conditional probability vector of spin i
    given values for all other spins ~i
    """
    pxi = np.zeros(ncolors)
    
    for color in range(ncolors):
        pxi[color] = J[i,i,color,color]
                
#         for spin in range(nspins-1):
#             if spin < i:
#                 pxi[color] += J[i,spin,color,x[spin]]
#             elif spin >= i:
#                 pxi[color] += J[i,spin+1,color,x[spin]]
                
        for spin in range(nspins):
            if spin!=i:
                #pxi[color] += J[i,spin,color,x[spin]]
                pxi[color] += J[i,spin,color,x[spin]] + J[spin,i,x[spin],color]
                
    #pxi = np.exp(beta*pxi)
    pxi = np.exp(beta*pxi/2)
    pxi = pxi/np.sum(pxi)
    
    return pxi 


@jit(nopython=True)
def GibbsSample(nsamples,J,nspins,ncolors,beta=1,burn=0):
    """
    Gibbs sample the Potts model specified by coupling matrix
    J at inverse temperature beta. Discards the first burn
    samples.
    Output: nsamples x nspins matrix
    """
    
    samples = np.empty((nsamples,nspins),dtype=np.int64)
    xj = np.random.choice(ncolors,nspins) #Random initial state
    
    j = 0
    
    while j<nsamples+burn:
        
        
        for i in range(nspins):
        
            pxi = conditionalPottsProb(i,xj,J,nspins,ncolors,beta)
            pxi = np.cumsum(pxi)
            r = np.random.rand()
            
            for color in range(ncolors):
                if r <= pxi[color]:
                    xj[i] = color
                    break
        
        if j>=burn:
            samples[j-burn,:] = xj[:]
        
        j += 1
    
    return samples

@jit(nopython=True)
def fullPottsProb(J,nspins,ncolors,beta=1):
    """
    Returns the full probability distribution for a 
    Potts model with specified J matrix
    """
    nstates = ncolors**nspins
    probabilities = np.zeros(nstates)
    
    config = np.zeros(nspins,dtype=np.int64)
    for state in range(nstates):
        
        if state==0:
            config = np.zeros(nspins,dtype=np.int64)
        else:
            num = state
            for spin in range(nspins):
                config[nspins-1-spin] = int(num%ncolors)
                num //= ncolors
        
        
#         for spin1 in range(nspins):
#             for spin2 in range(spin1,nspins):
#                 probabilities[state] += J[spin1,spin2,config[spin1],config[spin2]]

#         probabilities[state] = np.exp(beta*probabilities[state])
        
        
        for spin1 in range(nspins):
            for spin2 in range(nspins):
                probabilities[state] += J[spin1,spin2,config[spin1],config[spin2]]
        
        probabilities[state] = np.exp(beta*probabilities[state]/2)
    
    probabilities = probabilities/np.sum(probabilities)
        
    return probabilities

In [5]:
nspins = 8
ncolors = 2
nsamples = 1000
sparsity = 0.05


J = makeRandomCouplingMatrix(nspins,ncolors,sparsity)
samples = GibbsSample(nsamples,J,nspins,ncolors,beta=1,burn=1000)

np.save("gibbssamples.npy",samples)
np.save("gibbscoupling.npy",J)

samples_csv = np.insert(samples,0,np.ones(nsamples),axis=1)
np.savetxt('gibbssamples.csv',samples_csv,delimiter=",",fmt='%d')

In [6]:
print(J)

[[[[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.57200248]]

  [[ 0.          0.        ]
   [ 0.          0.97085711]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]]


 [[[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.4793922   0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]]


 [[[ 0.          0.        ]
   [ 0.